<a href="https://colab.research.google.com/github/nursena8/Rag-agentic-model-with-DeepSeek/blob/main/Rag_model_with_Deepseek_(agentic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
os.environ["HF_TOKEN"] = "your api key"

In [12]:
os.environ['TOGETHER_API_KEY'] = 'your api key'

In [1]:
!pip install langchain_community duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip install -U langchain-huggingface
!pip install faiss-gpu
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
#deepseek api
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0


In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain_community.tools import DuckDuckGoSearchResults
from together import Together
import pandas as pd
import os

In [15]:
df = pd.read_csv("movie.csv")
df =  df.sample(n=4000, random_state=42)
df['content'] = df['title'] + " " + df['genres']

In [17]:
client = Together()

def deepseek_llm(prompt):
    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V3",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

documents = [Document(page_content=row["content"]) for _, row in df.iterrows()]
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(documents, embedding_model)


In [ ]:
#definening tools without using crewAI

def rag_tool(query: str) -> str:
    """Search the local movie vector store for relevant content."""
    docs = vector_store.similarity_search(query, k=5)
    return "\n".join([doc.page_content for doc in docs])

def web_search_tool(query: str) -> str:
    """Search the internet for movie-related information using DuckDuckGo."""
    search = DuckDuckGoSearchResults()
    results = search.run(query)
    return "\n".join(results[:5])


In [ ]:
#definening agents

def router_agent_function(query: str) -> str:
    prompt = (
        f"Analyze the user query: {query} and determine the best search method.\n\n"
        "1. Use 'vectorstore' if:\n"
        "   - The question asks about specific movies, genres, or related metadata.\n"
        "   - The answer is likely available in our movie database.\n\n"
        "2. Use 'web_search' if:\n"
        "   - The question asks for real-time information (e.g., latest releases, awards, trending movies).\n"
        "   - The answer is not covered in our movie database.\n\n"
        "Return exactly one word: 'vectorstore' or 'web_search'."
    )
    return deepseek_llm(prompt).strip().lower()

def retriever_agent_function(query: str, route_output: str) -> str:
    if route_output == "vectorstore":
        return rag_tool(query)
    elif route_output == "web_search":
         return web_search_tool(query)
    else:
        return "Invalid route decision"

def answer_grader_agent_function(query: str, retrieved_info: str) -> str:
    prompt = (
        f"Evaluate if the retrieved content is relevant to the query: {query}.\n"
        f"Retrieved info: {retrieved_info} \n"
        "Return 'yes' if the response aligns with the query.\n"
        "Return 'no' if the response does not match the query."
    )
    return deepseek_llm(prompt).strip().lower()


def hallucination_grader_agent_function(query: str, retrieved_info: str) -> str:
        prompt = (
        f"Check if the retrieved response for query {query} contains factual information.\n"
        f"Retrieved info: {retrieved_info} \n"
        "Return 'yes' if the response is factually accurate.\n"
        "Return 'no' if the response contains incorrect or unsupported claims."
    )
        return deepseek_llm(prompt).strip().lower()

def answer_generator_agent_function(query: str, retrieved_info: str, hallucination_check: str) -> str:
    prompt = (
        f"Generate a final response for the query: {query}.\n"
        f"Use the validated information: {retrieved_info}.\n"
        f"Hallucination check result: {hallucination_check}\n"
        "If the hallucination check returns 'no', perform a web search for more accurate information.\n"
        "Return a clear, well-structured answer."
    )

    if hallucination_check == 'no':
          prompt += f" web search result: {web_search_tool(query)}\n"
    return deepseek_llm(prompt)


In [18]:
def get_movie_answer(user_query: str):
    print("User Query:", user_query)

    route_output = router_agent_function(user_query)
    print("Router Output:", route_output)

    retrieved_info = retriever_agent_function(user_query, route_output)
    print("Retrieved Info:", retrieved_info)

    answer_grade = answer_grader_agent_function(user_query, retrieved_info)
    print("Answer Grade:", answer_grade)

    hallucination_check = hallucination_grader_agent_function(user_query, retrieved_info)
    print("Hallucination Check:", hallucination_check)

    final_response = answer_generator_agent_function(user_query, retrieved_info, hallucination_check)
    print("Final Response:", final_response)

    return final_response



query1 = "What are the best sci-fi movies from the 1990s?"
response1 = get_movie_answer(query1)
print("Response 1:", response1)

User Query: What are the best sci-fi movies from the 1990s?
Router Output: vectorstore
Retrieved Info: Space Cowboys (2000) Action|Adventure|Comedy|Sci-Fi
Zapped Again! (1990) Comedy|Sci-Fi
Aliens (1986) Action|Adventure|Horror|Sci-Fi
Total Recall (1990) Action|Adventure|Sci-Fi|Thriller
Interstellar (2014) Sci-Fi|IMAX
Answer Grade: no. the retrieved content includes movies from various years, not exclusively from the 1990s. only *zapped again! (1990)* and *total recall (1990)* are relevant to the query, while the others are from different decades. therefore, the response does not fully align with the query.
Hallucination Check: the response contains several inaccuracies and unsupported claims:

1. **space cowboys (2000)** - this movie is not from the 1990s.
2. **aliens (1986)** - this movie is from the 1980s, not the 1990s.
3. **interstellar (2014)** - this movie is from the 2010s, not the 1990s.

only **zapped again! (1990)** and **total recall (1990)** are correctly identified as bei